In [36]:
import cv2
import os
import numpy as np

single_data = 1
multi_data = 5
more_data = 20
full_data = 500

current_data = more_data

# Dataset Path
dataset_path = "../dataset/train/"

# Padding Removed Directory
dir_padding_removed = "padding_removed"
# HSV Converted Directory
dir_hsv_converted = "hsv_converted"
# K Means Segmented Directory
dir_kmeans_segmented = "kmeans_segmented"
# Processed Foregrounds Directory
dir_processed_foregrounds = "processed_foregrounds"
# Resulting Detected Objects
dir_result = "detected_objectsv1"


# Remove all white pixels

In [37]:
def removeWhite(image):
    image[np.where((image==[255,255,255]).all(axis=2))] = [0,0,0]
    return image

# Simple Remove White Padding Code

In [38]:
if os.path.exists(dir_padding_removed) and os.path.isdir(dir_padding_removed):
    pass
else: 
    os.mkdir(dir_padding_removed)

# # Change this to process more images
for i in range(current_data):
    image_path = dataset_path + f"image_id_{i:03d}.jpg"
    output_path = dir_padding_removed + f"/result{i:03d}.png"
    training_image = cv2.imread(image_path)
    # output = blackenPads(training_image)
    # output, pads = blackenPadsSimple(training_image)
    # output = removePads(training_image)
    output = removeWhite(training_image)
    cv2.imwrite(output_path, output)

# HSV Generator
- Output: (hsv_padding_removed)

In [39]:
# Blurring & Generating HSVs - Gabriel Thien - Computer Visionaries

input_directory_path = dir_padding_removed
output_directory_path = dir_hsv_converted

if os.path.exists(output_directory_path) and os.path.isdir(output_directory_path):
    pass
else: 
    os.mkdir(output_directory_path)

for i in range(current_data):
    imagePath = input_directory_path + f"/result{i:03d}.png"
    input_image = cv2.imread(imagePath)
    hsv_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2HSV)
    
    # Blurring
    kernel_size_mean = (5, 5)
    hsv_blurred = cv2.blur(hsv_image, kernel_size_mean)
    
    # cv2.imshow("Blurrer HSV", hsv_blurred)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    # cv.waitKey(1)
    
    cv2.imwrite(output_directory_path + f"/result{i:03d}.png", hsv_blurred)

# K Means Segmented
- Output: hsv_kmeans_segmented

In [40]:
# Colour Segmentation - Gabriel Thien - Computer Visionaries

import cv2
import numpy as np
from sklearn.cluster import KMeans

def makeFlattenedMatrix(image):
    pixels = []
    
    for row in image:
        for rgb_pixel in row:
            pixels.append((rgb_pixel[0], rgb_pixel[1], rgb_pixel[2]))

    return np.array(pixels)    
    

def kMeansSegment(imagePath):
    image = cv2.imread(imagePath)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    pixels = makeFlattenedMatrix(image_rgb)

    # K means clustering
    K = 3
    n = 2
    # Perform K-means clustering
    kmeans = KMeans(n_init=n, n_clusters=K, random_state=0)
    kmeans.fit(pixels)

    cluster_centers = kmeans.cluster_centers_
    labels = kmeans.labels_
    # print(cluster_centers)
    # print(labels)

    # Create the segmented image
    segmented_image = cluster_centers[labels].reshape(image.shape).astype(np.uint8)
    
    return segmented_image

# cv2.waitKey(0)
# cv2.destroyAllWindows()
# cv2.waitKey(1)

input_directory_path = dir_hsv_converted
output_directory_path = dir_kmeans_segmented

if os.path.exists(output_directory_path) and os.path.isdir(output_directory_path):
    pass
else: 
    os.mkdir(output_directory_path)

for i in range(current_data):
    imagePath = input_directory_path + f"/result{i:03d}.png"
    result = kMeansSegment(imagePath)
    cv2.imwrite(output_directory_path + f"/kmeans_result{i:03d}.png", result)

# Determining colour frequencies of a segmented image

In [41]:
# Computing Colours - Gabriel Thien - Computer Visionaries

import matplotlib.pyplot as plotter
import numpy as np


def computeFrequencies(image):
    a_f = [0 for i in range(256)]
    b_f = [0 for i in range(256)]
    c_f = [0 for i in range(256)]
    for row in image:
        for pixel in row:
            # print(pixel)
            a_f[pixel[0]] += 1
            b_f[pixel[1]] += 1
            c_f[pixel[2]] += 1

    return a_f, b_f, c_f


def computePixelFrequencies(image) -> list:
    pixels = {}
    for row in image:
        for pixel in row:
            if tuple(pixel) in pixels.keys():
                pixels[tuple(pixel)] += 1
            else:
                pixels[tuple(pixel)] = 1

    return sorted(pixels.items(), key=lambda x: x[1])


def removePixel(image, targetPixel: tuple):
    output = image
    num_rows = image.shape[0]
    num_cols = image.shape[1]

    for x in range(num_rows):
        for y in range(num_cols):
            pixel = tuple(image[x][y])
            if pixel == targetPixel:
                output[x][y] = (0, 0, 0)
    return output


def removeNoise(image, colours: list):
    num_rows = image.shape[0]
    num_cols = image.shape[1]

    # black_image = np.zeros((num_rows, num_cols, 3), dtype=np.uint8)

    image_copy = image.copy()

    # Colours - THREE
    colour_low = tuple(colours[0][0])
    colour_mid = tuple(colours[1][0])
    colour_high = tuple(colours[2][0])
    # print(colour_low)

    # Colours - TWO
    # colour_low = tuple(colours[0][0])
    # colour_high = tuple(colours[1][0])

    # K1
    kernel = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]], dtype=np.uint8)

    # K2
    # kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]], dtype=np.uint8)

    # K3
    # kernel = np.ones((3, 3), np.uint8)

    # Original
    # cv2.imshow("Original", image)

    # Foreground stuff
    foreground_colour = preserveSingleColour(image_copy, colour_low)
    foreground_eroded = cv2.erode(foreground_colour, kernel, iterations=15)
    foreground_focused_image = cv2.dilate(foreground_eroded, kernel, iterations=15)

    # # Midground stuff
    # midground_colour = preserveSingleColour(image_copy, colour_mid)

    # Background stuff
    # background_colour = preserveSingleColour(image_copy, colour_high)
    # background_dilated = cv2.dilate(background_colour, kernel, iterations=20)
    # background_eroded = cv2.erode(background_dilated, kernel, iterations=20)
    # background_focused_image = cv2.bitwise_not(background_eroded)

    # cv2.imshow("Foreground Colour", foreground_colour)
    # cv2.imshow("Eroded", foreground_eroded)
    # cv2.imshow("Foreground", foreground_focused_image)

    # cv2.imshow("Background Colour", background_colour)
    # cv2.imshow("Background Dilated", background_dilated)
    # cv2.imshow("Background Eroded", background_eroded)
    # cv2.imshow("Background", background_focused_image)

    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # cv2.waitKey(1)
    return foreground_focused_image


def preserveSingleColour(image, targetPixel: tuple):
    image_copy = image.copy()

    # Pixel mask that isnt the target pixel
    mask = np.all(image_copy != targetPixel, axis=2)
    # Applies mask
    image_copy[mask] = [0, 0, 0]

    # cv2.imshow("Grayscale single channel", grayscale)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # cv2.waitKey(1)
    return image_copy


def defaultBackground(image, colours: tuple, tolerance=10):
    num_rows = image.shape[0]
    num_cols = image.shape[1]

    most_common_colour = colours[-1][0]

    for x in range(num_rows):
        for y in range(num_cols):
            pixel = image[x][y]
            if (
                pixel[0] <= tolerance
                and pixel[1] <= tolerance
                and pixel[2] <= tolerance
            ):
                image[x][y] = most_common_colour

    return image

input_directory_path = dir_kmeans_segmented
output_directory_path = dir_processed_foregrounds

if os.path.exists(output_directory_path) and os.path.isdir(output_directory_path):
    pass
else: 
    os.mkdir(output_directory_path)

for i in range(current_data):
    imagePath = input_directory_path + f"/kmeans_result{i:03d}.png"
    image = cv2.imread(imagePath)

    b_f, g_f, r_f = computeFrequencies(image)
    pixel_frequencies = computePixelFrequencies(image)
    # print(pixel_frequencies)

    most_frequent_pixel = pixel_frequencies[-1][0]

    defaulted_background = defaultBackground(image, pixel_frequencies)

    # processed_image = removePixel(defaulted_background, most_frequent_pixel)

    processed_image = removeNoise(defaulted_background, pixel_frequencies)

    cv2.imwrite(output_directory_path + f"/result{i:03d}.png", processed_image)

    # cv2.imshow(f"Processed Image", processed_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # cv2.waitKey(1)

    # plotter.plot(b_f, color="blue")
    # plotter.plot(g_f, color="green")
    # plotter.plot(r_f, color="red")
    # plotter.show()


# Drawing Bounding Boxes

In [42]:
# Draw Bounding Boxes - Gabriel Thien - Computer Visionaries
# Image has been segmented and the outline of the image is ready to be highlighted

def getContours(image):
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    _, thresholded = cv2.threshold(grayscale, 0, 255, cv2.THRESH_BINARY)
    
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # print(f"Type of contours: {type(contours)}")
    return contours

# All areas
def calculateBoundingBox(contours): 
    x_min = x_max = 0
    y_min = y_max = 0
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if (x_min == 0) and (x_max == 0) and (y_min == 0) and (y_max == 0):
            x_min = x
            x_max = x + w
            y_min = y
            y_max = y + h
        if (x < x_min): x_min = x
        elif (x + w > x_max): x_max = x + w
        if (y < y_min): y_min = y
        elif (y + h > y_max): y_max = y + h
        
    return (x_min, y_min), (x_max, y_max)

def drawBoundingBox(island_image, original_image):
    # Get contours
    contours = getContours(island_image)
    # Get the coordinates of the bounding box to be drawn
    top_left_corner, bottom_right_corner = calculateBoundingBox(contours)
    # Draw
    drawn = cv2.rectangle(original_image, top_left_corner, bottom_right_corner, (0, 255, 0), 2)
    return drawn

input_directory_path = dir_processed_foregrounds
output_directory_path = dir_result

if os.path.exists(output_directory_path) and os.path.isdir(output_directory_path):
    pass
else: 
    os.mkdir(output_directory_path)

for i in range(current_data):
    islandImagePath = input_directory_path + f"/result{i:03d}.png"
    island_image = cv2.imread(islandImagePath)
    
    originalImagePath = dataset_path + f"image_id_{i:03d}.jpg"
    original_image = cv2.imread(originalImagePath)
    
    drawn_image = drawBoundingBox(island_image, original_image)
        
    cv2.imwrite(output_directory_path + f"/detectedv1_{i:03d}.png", drawn_image)
    # cv2.imshow("Drawn Image", drawn_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # cv2.waitKey(1)